In [2]:
import requests
import re
from bs4 import BeautifulSoup
import sqlite3

In [3]:
def get_products(page, database):
    soup = BeautifulSoup(page.text, 'html.parser')
    items = soup.find_all('a', attrs={'class':['c-product-box__img','c-promotion-box__image','js-url','js-product-item','js-product-url']})
    for item in items:
        text = ''
        found = re.search(r'product_url\":\"\/product\/[a-z,A-Z,0-9,-]+',str(item))
        if found is not None: 
            text += found.group(0)
            text = re.sub(r'product_url\":\"\/product\/','',text)
            database.execute("INSERT INTO products VALUES ('%s', 0)"%text)
            database.commit()

In [ ]:
print('Connecting to database...')
database = sqlite3.connect('list.db')
cur = database.cursor()
try:
    cur.execute("""CREATE TABLE products(
        product TEXT,
        done INTEGER
    )
    """)
    database.commit()
except:
        print('connected.')
print('Extracting list of products...')
for i in range(1, 10):
    url = 'https://www.digikala.com/search/category-notebook-netbook-ultrabook/?has_selling_stock=1&pageno={}&sortby=21'.format(i)
    page = requests.get(url)
    if page.status_code == 200: get_products(page, database)
    else: print('Failed! url = {}'.format(url))